<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/last_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
!pip install tqdm
import rasterio
from rasterio.plot import reshape_as_image
from tqdm import tqdm

!pip install torchgeo
import torchgeo.models

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

     |████████████████████████████████| 19.3 MB 9.7 MB/s 
     |████████████████████████████████| 204 kB 8.1 MB/s 
     |████████████████████████████████| 584 kB 56.5 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
     |████████████████████████████████| 1.0 MB 48.0 MB/s 
     |████████████████████████████████| 409 kB 64.1 MB/s 
     |████████████████████████████████| 493 kB 50.8 MB/s 
     |████████████████████████████████| 431 kB 62.5 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
     |████████████████████████████████| 16.7 MB 46.5 MB/s 
     |████████████████████████████████| 6.3 MB 52.5 MB/s 
     |████████████████████████████████| 112 kB 61.0 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 136 kB 54.0 MB/s 
     |████████████████████████████████| 1.1 MB 51.5 MB/s 
     |████████████████████████████████| 376 kB 51.6 MB/s 
     |████████████████████████████████| 58 kB 6.5 MB/s 
     |██████████████

In [3]:
#local
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#folder = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [4]:
#Define paths

ROOT_PATH = '/content/drive/MyDrive/ML/'
#ROOT_PATH = '/Volumes/GoogleDrive/My Drive/ML'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
#file_path_train = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/train.csv'
#file_path_test = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [5]:
data_df_train.head()

,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [6]:
type(data_df_train)

pandas.core.frame.DataFrame

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [8]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [9]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [10]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [11]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [12]:
#Reload the .npy files

import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#path = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

100%|██████████| 10/10 [00:42<00:00,  4.22s/it]


In [13]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [159]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [160]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_ds = EuroSAT(data_df_train, BASE_PATH, train_transform)
print(len(train_ds))

27000


In [161]:
train_size = int(0.8 * len(data_df_train))
test_size = len(data_df_train) - train_size
train_ds, validation_ds = torch.utils.data.random_split(train_ds, [train_size, test_size], generator=torch.Generator().manual_seed(seed_value))

In [162]:
print(len(train_ds))
print(len(validation_ds))

21600
5400


In [250]:
model = models.efficientnet_b0(pretrained=True)

Model

In [260]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [252]:
pretrainedBool = True
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
model.classifier = nn.Sequential(
                      nn.Linear(2048, 256),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(256, 10),
                      nn.LogSoftmax(dim=1)
                        )

In [253]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [254]:
model = model.to(device)

In [255]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [256]:
ce_loss = ce_loss.to(device)

In [245]:
config={
"epochs": 20,
"mini_batch_size": 64,
"learning_rate": 0.01,
"opt_func": torch.optim.Adam,
"milestones": [4,8,15],
"weight_decay": 0,
"model": "efficientnet_b7",
"transform": "no_random_rotation",
"bands":"RGB and all from ResNet"
}

In [246]:
train_dl = DataLoader(train_ds, batch_size=config.get("mini_batch_size"), shuffle=True)

In [234]:
#Empty cache
torch.cuda.empty_cache()

In [ ]:
!pip install wandb
!wandb login
import wandb
#Key: 9da448bfaa162b572403e1551114a17058f249d0

In [247]:
run = wandb.init(project="ml-challenge", entity="nicikess", config=config)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 996, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 168, in setup
    tel.feature.set_init_tags = True
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/telemetry.py", line 41, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 577, in _telemetry_callback
    self._telemetry_flush()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 588, in _telemetry_flush
    self._backend.interface._publish_telemetry(self._telemetry_obj)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 74, in _publish_telemetry
    self._publish(rec)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_queue.py", line 49, in _publish
    raise Exception("The wa

Exception: ignored

In [248]:
epochs = config.get("epochs")
learning_rate = config.get("learning_rate")
weight_decay = config.get("weight_decay")
opt_func = config.get("opt_func")
milestones = config.get("milestones")
max_learning_rate = config.get("max_learning_rate")

train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay)

Exception: ignored

In [223]:
def train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay):
  
  # init collection of training epoch losses
  train_epoch_losses = []

  # set the model in training mode
  model.train()

  optimizer = opt_func(model.fc.parameters(),lr=learning_rate, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

  # train the CIFAR10 model
  for epoch in range(epochs):
      
      # init collection of mini-batch losses
      train_mini_batch_losses = []

      # iterate over all-mini batches
      for i, (images, labels) in enumerate(train_dl):

          #print(images)

          # push mini-batch data to computation device
          images = images.to(device)
          labels = labels.to(device)

          # run forward pass through the network
          output = model(images)

          # reset graph gradients
          optimizer.zero_grad()
          
          # determine classification loss
          loss = ce_loss(output, labels)

          wandb.log({"loss": loss})
          
          # run backward pass
          loss.backward()
          
          # update network paramaters
          optimizer.step()
          
          # collect mini-batch reconstruction loss
          train_mini_batch_losses.append(loss.data.item())

      # determine mean min-batch loss of epoch
      train_epoch_loss = np.mean(train_mini_batch_losses)

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      scheduler.step()

      print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

      #Added
      #result = evaluate(model, vali_dataloader)

      # print epoch loss
      now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
      print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
      
      #if (epoch%10 == 0): 
          # set filename of actual model
      model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))
          # save current model to GDrive models directory
      torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))

      # determine mean min-batch loss of epoch
      train_epoch_losses.append(train_epoch_loss)

In [62]:
resnet50_vali_dataloader = DataLoader(validation_ds, batch_size=5400)

In [63]:
# Display image and label.
validation_features, validation_labels = next(iter(resnet50_vali_dataloader))
print(f"Labels batch shape: {validation_labels.size()}")

KeyboardInterrupt: ignored

In [136]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_9.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [ ]:
# set model in evaluation mode
best_model.eval()

In [141]:
type(predictions)

torch.Tensor

In [59]:
#Validation
#next() returns a tupel: tensor,label
predictions = torch.argmax(best_model(iter(resnet50_vali_dataloader).next()[0].to(device)), dim=1)

RuntimeError: ignored

In [138]:
predictionsVal = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(validation_ds))):
  input = validation_ds[i]
  input = input[0].unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictionsVal.append(yhat.cpu())
predictionsVal = vstack(predictionsVal)
predictionsVal = torch.tensor(predictionsVal)

100%|██████████| 5400/5400 [01:26<00:00, 62.14it/s]


In [140]:
list_tens = list()

for i in range(len(predictionsVal)):
  list_tens.append(decode_target(predictionsVal[i].item()))

In [141]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['HerbaceousVegetation', 'Industrial', 'River', 'AnnualCrop', 'Highway', 'PermanentCrop', 'SeaLake', 'Residential', 'Forest', 'Pasture'])
dict_values([656, 496, 502, 599, 475, 522, 565, 620, 580, 385])


In [142]:
from sklearn import metrics
acc = metrics.accuracy_score(validation_labels.detach().cpu(), predictionsVal.detach().cpu())
print(acc)

0.9451851851851852


In [ ]:
wandb.log({"acc": acc})

In [ ]:
#Finish run
run.finish()

In [42]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

100%|██████████| 4232/4232 [01:02<00:00, 67.48it/s] 


In [43]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, [3,2,1]]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [83]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    #transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [84]:
#Test data
#transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_ds = EuroSATTest(data_df_test, BASE_PATH, test_transform)
print(len(test_ds))

4232


In [ ]:
test_ds[0]

In [ ]:
img = datatest[3]
normalized_img = normalize_for_display(img)
fig, axs = plt.subplots(4,4, figsize=(15,15))

band_idx = 0
bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12", "B13"]

for i in range(4):
    for j in range(4):
        if band_idx < 13:
            axs[i,j].imshow(normalized_img[:, :, band_idx], cmap="gray")
            axs[i,j].set_title(bands[band_idx])
        else:
            axs[i,j].imshow(np.ones((64,64,3)), cmap="gray")
        
        axs[i,j].axis(False)
        band_idx += 1
        
plt.tight_layout()
plt.show()

In [67]:
resnet50_test_dataloader = DataLoader(test_ds, batch_size=5000)

In [41]:
validation_features = next(iter(resnet50_test_dataloader))

In [44]:
type(validation_features[0])

torch.Tensor

In [259]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_13.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

RuntimeError: ignored

In [ ]:
# set model in evaluation mode
best_model.eval()

In [68]:
#predictions = evaluate_modelTest(resnet50_test_dataloader,best_model)
predictions = torch.argmax(best_model(iter(resnet50_test_dataloader).next().to(device)), dim=1)

RuntimeError: ignored

In [257]:
predictions = list()
from PIL import Image
from numpy import vstack
for i in tqdm(range(len(test_ds))):
  input = test_ds[i]
  input = input.unsqueeze(0)
  #print(input)
  #input = test_transform(input)
  #x = x[None, :]
  input = input.to(device)
  #print(type(input))
  yhat = best_model(input)
  # retrieve numpy array
  yhat = yhat.detach()
  # convert to class labels
  yhat = torch.argmax(yhat, axis=1)
  # reshape for stacking
  yhat = yhat.reshape((len(yhat), 1))
  # store
  predictions.append(yhat.cpu())
predictions = vstack(predictions)
predictions = torch.tensor(predictions)

  7%|▋         | 280/4232 [00:05<01:11, 55.46it/s]


KeyboardInterrupt: ignored

In [ ]:
#Test
#predictions = torch.argmax(best_model(iter(resnet50_test_dataloader).next()), dim=1)

In [89]:
list_tens = list()

for i in range(len(predictions)):
  list_tens.append(decode_target(predictions[i].item()))

In [90]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['River', 'SeaLake', 'Highway', 'Industrial', 'HerbaceousVegetation', 'AnnualCrop', 'PermanentCrop', 'Residential', 'Pasture', 'Forest'])
dict_values([957, 1067, 703, 277, 284, 307, 267, 160, 139, 71])


In [ ]:
from sklearn import metrics
metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

0.8346296296296296

In [54]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission_new.csv', index=None)